# **sempy_functions**

Set of functions developed to carry out functionalities using the [Semantic-Link](https://pypi.org/project/semantic-link/) library.

#### **🧷 Imports and references 🧷**

---

In [ ]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import time, json

#### **🛠️ Fabric Utils 🛠️**

Functions designed to facilitate usability and carry out actions within Fabric in a simplified way.

1. **fabric_rest_api_caller**, wrapper for Fabric REST API calls through SemPy
1. **sql_endpoint_refresh**, function for refreshing SQL endpoint data
1. **semantic_model_refresh**, refresh semantic models through SemPy

You can read more about this methods on [Medium](https://medium.com/@baggirraf/cada-cosa-a-su-tiempo-sincronizaci%C3%B3n-y-refresco-de-modelos-sem%C3%A1nticos-con-microsoft-fabric-b6efec06185e).

</br>

---

In [ ]:
def fabric_rest_api_caller(
    source_uri: str, method: str, source_payload: str = None
) -> dict:
    """
    Author : Rafael Báguena Girbés  |   02-13-2025

    Function to facilitate making Fabric API calls through the sempy library.

    Receives the desired method for the call, its url and the content to be included if necessary.

    Returns the received response for further use.

    args:   source_uri
            URL of the desired API call

            method
            Method used in the call (post or get)

            source_payload
            Content to be included in the call, may be omitted
    """
    try:
        # Initialize the client
        fabric_client: any = fabric.FabricRestClient()
        # Check arguments received and return response
        match (method):
            case "post":
                if source_payload:
                    response = fabric_client.post(source_uri, json=source_payload)
                else:
                    response = fabric_client.post(source_uri)
            case "get":
                response = fabric_client.get(source_uri)
            case _:
                raise NotImplementedError
        return response
    except NotImplementedError:
        print("*ERROR*: ❌ Specified method is not implemented.")
    except Exception as err:
        print(f"*ERROR*: ❌ Something went wrong, the error ocurred is: {err=}")

def sql_endpoint_refresh(sql_endpoint_id: str) -> None:
    """
    Author : Rafael Báguena Girbés  |   02-13-2025

    Function that enables refreshing of the received SQL endpoint via API call.

    No need to specify the workspace.

    Once the refresh request is triggered, its status is checked every 30 seconds to ensure that it is complete.

    args:   sql_endpoint_id
            Identifier of the SQL endpoint to be refreshed
    """
    # Declare variables
    call_uri: str = f"/v1.0/myorg/lhdatamarts/{sql_endpoint_id}"
    payload: dict = {"commands": [{"$type": "MetadataRefreshCommand"}]}

    try:
        print(f"*INFO*: ▶️ Starting refresh for '{sql_endpoint_id}' SQL Endpoint.")
        # Request for refreshment and process response
        refresh_response = fabric_rest_api_caller(call_uri, "post", payload)
        batch_id_value: str = refresh_response.json()["batchId"]
        progress_status: str = refresh_response.json()["progressState"]

        # Check request progress each 30 seconds until status changes
        status_uri: str = (
            f"/v1.0/myorg/lhdatamarts/{sql_endpoint_id}/batches/{batch_id_value}"
        )
        while progress_status == "inProgress":
            print(f"*INFO*: 🔎 Checking status for '{batch_id_value}' batch.")
            status_response = fabric_rest_api_caller(status_uri, "get")
            progress_status: str = status_response.json()["progressState"]
            print(f"*INFO*: ⚠️ Current status is '{progress_status}'.")
            match(progress_status):
                case "success":
                    break
                case "failed":
                    raise Exception("The refresh has failed.")
                case _:
                    print("*INFO*: ⌛ Checking again in 30 seconds.")
                    time.sleep(30)

        print(f"*INFO*: ✅ SQL Endpoint '{sql_endpoint_id}' successfully refreshed.")
    except Exception as err:
        print(f"*ERROR*: ❌ Something went wrong, the error ocurred is: {err=}")

def semantic_model_refresh(workspace_id: str, dataset_id: str) -> None:
    """
    Author : Rafael Báguena Girbés  |   02-13-2025

    This function executes the refresh of the specified semantic model thanks to the existing sempy functions refresh_dataset and get_refresh_execution_details.

    Once the refresh request is triggered, its status is checked every 30 seconds to ensure that it is complete.

    args:   workspace_id
            Identifier of the workspace where the semantic model resides

            dataset_id
            Identifier of the semantic model to be refreshed
    """
    try:
        print(f"*INFO*: ▶️ Starting refresh for '{dataset_id}' semantic model.")
        # Request for refreshment
        refresh_request_id = fabric.refresh_dataset(
            dataset=dataset_id, workspace=workspace_id
        )

        # Check request progress each 30 seconds until status changes
        refresh_status: str = "Not started"
        while refresh_status not in {"Completed", "Failed"}:
            refresh_details = fabric.get_refresh_execution_details(
                dataset=dataset_id,
                refresh_request_id=refresh_request_id,
                workspace=workspace_id,
            )
            refresh_status = refresh_details.status
            print(f"*INFO*: ⚠️ Current status is '{refresh_status}'.")
            match (refresh_status):
                case "Completed":
                    break
                case "Failed":
                    raise Exception("The refresh has failed.")
                case _:
                    print("*INFO*: ⌛ Checking again in 30 seconds.")
                    time.sleep(30)

        print(f"*INFO*: ✅ Semantic model '{dataset_id}' successfully refreshed.")
    except Exception as err:
        print(f"*ERROR*: ❌ Something went wrong, the error ocurred is: {err=}")

#### **📞 Fabric REST API calls 📞**

Using SemPy to perform actions through [Fabric REST API](https://learn.microsoft.com/en-us/rest/api/fabric/articles/) methods. 

Each function receives, at a minimum, the payload that will accompany the request to provide the elements it requires. In case a workspace identifier is not provided in the call, the one corresponding to the workspace where the notebook resides is used.

The response code returned is checked for errors in the request.

---

##### 1️⃣ **Element creation**

1. **create_workspace**, based on the [Workspaces - Create Workspace](https://learn.microsoft.com/en-us/rest/api/fabric/core/workspaces/create-workspace?tabs=HTTP) method
1. **create_fabric_artifact**, based on the [Items - Create Item](https://learn.microsoft.com/en-us/rest/api/fabric/core/items/create-item?tabs=HTTP) method
1. **create_shortcut**, based on the [OneLake Shortcuts - Create Shortcut](https://learn.microsoft.com/en-us/rest/api/fabric/core/onelake-shortcuts/create-shortcut?tabs=HTTP) method

</br>

---

In [ ]:
def create_workspace(payload:dict):
    """
    Author : Rafael Báguena Girbés  |   05-19-2024

    This function performs the creation of a work area within the Power BI service. Through the properties included within the payload, 
    it can be automatically assigned to an existing capability.

    To be able to create work areas and/or assign them to a capability, you must have permission to do so.

    Returning the response is useful to get the identifier for the new workspace.

    args:   payload
            Dictionary for the request body specifications, displayName is required
            Sample:
                {
                    "displayName": "WORKSPACE_NAME",
                    "capacityId": "CAPACITY_IDENTIFIER",
                    "description": "WORKSPACE_DESCRIPTION"
                }
    """
    try:
        # Initialize the client
        fabric_client = fabric.FabricRestClient()

        # Make the request and store the response
        response = fabric_client.post(f"/v1/workspaces", json= payload)

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)

        # Display information about the action performed
        print(f"*INFO*: Workspace '{payload.get('displayName')}' successfully created, its identifier is: {response.json().get('id')}")
        print("*INFO*: Remember to refresh your browser so that it appears in the Workspace menu")
        return response

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)
        return None
    
def create_fabric_artifact(payload:dict, workspace=None):
    """
    Author : Rafael Báguena Girbés  |   05-19-2024

    This function performs the creation of an artifact within the Power BI service. Through the properties specified in the request, 
    the type of object and its characteristics are defined.

    The caller must have contributor or higher workspace role. Fabric artifacts cannot be created on non-capacity workspaces.

    args:   payload
                Dictionary for the request body specifications, displayName and type are required
                Sample:
                    {
                        "displayName": "ARTIFACT_NAME",
                        "type": "ARTIFACT_TYPE",
                        "description": "ARTIFACT_DESCRIPTION"
                    }

            workspace
                Workspace name, if omitted the value of the workspace where the notebook resides is retrieved
    """
    # Check if workspace was provided, if not get current
    if workspace is None:
        workspace_id = fabric.get_workspace_id()
    else:
        workspace_id = fabric.resolve_workspace_id(workspace)

    try:
        # Initialize the client
        fabric_client = fabric.FabricRestClient()

        # Make the request and store the response
        response = fabric_client.post(f"/v1/workspaces/{workspace_id}/items",json= payload)

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)

        # Display information about the action performed
        print(f"*INFO*: {payload.get('type')} '{payload.get('displayName')}' successfully created")
        print(f"*INFO*: Remember to refresh the browser if it does not appear in the workspace: '{workspace}'")

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)

def create_shortcut(payload:dict, workspace:str, item:str):
    """
    Author : Rafael Báguena Girbés  |   05-19-2024

    This function creates a shortcut within the specified lakehouse to connect against the target.
    
    The parameters of the request change depending on the target type.

    args:   payload
                Dictionary for the request body specifications, name, path and target are required
                Sample for OneLake shorcut
                {
                    "path": "SOURCE_PATH",
                    "name": "SHORTCUT_NAME",
                    "target": 
                    {
                        "oneLake": 
                        {
                            "workspaceId": "TARGET_WORKSPACE_IDENTIFIER",
                            "itemId": "TARGET_ITEM_IDENTIFIER",
                            "path": "TARGET_PATH"
                        }
                    }
                }

            workspace
                Workspace identifier, if omitted the value of the workspace where the notebook resides is retrieved

            item
                Identifier of the artifact on which the shortcut is to be created
    """
    try:
        # Check if workspace was provided, if not get current
        if workspace is None:
            workspace_id = fabric.get_workspace_id()
        else:
            workspace_id = fabric.resolve_workspace_id(workspace)

       # Initialize the client
        fabric_client = fabric.FabricRestClient()

        # Make the request and store the response
        response = fabric_client.post(f"/v1/workspaces/{workspace_id}/items/{item}/shortcuts", json= payload)

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)

        # Display information about the action performed
        print(f"*INFO*: Shortcut '{payload.get('name')}' successfully created on the artifact '{item}'")
        print("*INFO*: Remember to refresh the contents of the lakehouse if it does not appear")

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)

##### 2️⃣ **Git operations**

1. **connect_git_repository**, based on the [Git - Connect](https://learn.microsoft.com/en-us/rest/api/fabric/core/workspaces/create-workspace?tabs=HTTP) method
1. **initialize_connection**, based on the [Git - Initialize Connection](https://learn.microsoft.com/en-us/rest/api/fabric/core/items/create-item?tabs=HTTP) method
1. **update_from_git_repository**, based on the [Git - Update From Git](https://learn.microsoft.com/en-us/rest/api/fabric/core/onelake-shortcuts/create-shortcut?tabs=HTTP) method

In case you want to perform the connection and subsequent dump of content from the repository to the workspace, the functions must be executed in the specified order.

The initialization of the connection provides the values necessary to carry out the update.

</br>

---

In [ ]:
def connect_git_repository(payload:dict, workspace=None):
    """
    Author : Rafael Báguena Girbés  |   05-19-2024

    This function connects a specific workspace to a git repository and branch. This operation does not sync between the workspace and the connected branch. 
    To complete the sync, use the initialize_connection function and follow with update_from_git_repository function.
    
    The caller must have an admin role for the workspace.

    args:   payload
                Dictionary for the request body specifications, gitProviderDetails are required
                Sample
                {
                    "gitProviderDetails": 
                    {
                        "organizationName": "ORGANIZATION_NAME",
                        "projectName": "PROJECT_NAME",
                        "gitProviderType": "AzureDevOps",
                        "repositoryName": "REPO_NAME",
                        "branchName": "BRANCH_NAME",
                        "directoryName": "DIRECTORY_NAME"
                    }
                }

            workspace
                Workspace identifier, if omitted the value of the workspace where the notebook resides is retrieved
    """
    try:
        # Check if workspace was provided, if not get current
        if workspace is None:
            workspace_id = fabric.get_workspace_id()
        else:
            workspace_id = fabric.resolve_workspace_id(workspace)

        # Initialize the client
        fabric_client = fabric.FabricRestClient()

        # Make the request and store the response
        response = fabric_client.post(f"/v1/workspaces/{workspace_id}/git/connect", json=payload)

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)

        # Display information about the action performed
        print(f"*INFO*: Workspace '{workspace_id}' connected to the repository '{payload['gitProviderDetails']['repositoryName']}' successfully")
        print(f"*INFO*: Remember to initialize the connection before downloading the repository contents")

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)

def initialize_connection(payload:dict, workspace=None):
    """
    Author : Rafael Báguena Girbés  |   05-19-2024

    This function intializes the connection for a workspace that is connected to Git. It should be called after a successful call to the previous function. 
    To complete a full sync of the workspace, use the Required Action operation returned in the response to call the relevant sync operation.
    
    The caller must have an admin role for the workspace.

    Returning the response is useful to get values such as the required action, the remote commit hash for the repo and/or the workspace head value,
    used in the next function.

    args:   payload
                Dictionary for the request body specifications, InitializationStrategy is required
                Sample
                {
                    "initializationStrategy": "DESIRED_INITIALIZATION_STRATEGY"
                }

            workspace
                Workspace identifier, if omitted the value of the workspace where the notebook resides is retrieved
    """    
    try:
        # Check if workspace was provided, if not get current
        if workspace is None:
            workspace_id = fabric.get_workspace_id()
        else:
            workspace_id = fabric.resolve_workspace_id(workspace)

        # Initialize the client
        fabric_client = fabric.FabricRestClient()

        # Make the request and store the response
        response = fabric_client.post(f"/v1/workspaces/{workspace_id}/git/initializeConnection", json=payload)

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)

        # Display information about the action performed
        if response.status_code == 200:
            print(f"*INFO*: Initialized connection to the repository correctly, the required action is: {response.json().get('requiredAction')}")
        else:
            print(f"*INFO*: Initialized connection to the repository correctly")
        print(f"*INFO*: You can now proceed with the content update")
        return response

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)
        return None

def update_from_git_repository(payload:dict, workspace=None):
    """
    Author : Rafael Báguena Girbés  |   05-19-2024

    This function updates the workspace with commits pushed to the connected branch. The update only affects items in the workspace that were changed in those commits. 
    If called after the connect_git_repository and initialize_connection functions, it will perform a full update of the entire workspace content.
    
    The caller must have a contributor or higher role for the workspace.

    args:   payload
                Dictionary for the request body specifications
                Sample
                {
                    "workspaceHead": "WORKSPACE_HEAD_VALUE",
                    "remoteCommitHash": "REMOTE_COMMIT_HASH_VALUE",
                    "conflictResolution": {
                        "conflictResolutionType": ""Workspace"",
                        "conflictResolutionPolicy": "PreferRemote"
                    },
                    "options": {
                        "allowOverrideItems": True
                    }
                }

            workspace
                Workspace identifier, if omitted the value of the workspace where the notebook resides is retrieved
    """
    try:
        # Check if workspace was provided, if not get current
        if workspace is None:
            workspace_id = fabric.get_workspace_id()
        else:
            workspace_id = fabric.resolve_workspace_id(workspace)

        # Initialize the client
        fabric_client = fabric.FabricRestClient()

        # Make the request and store the response
        response = fabric_client.post(f"/v1/workspaces/{workspace_id}/git/updateFromGit", json=payload)

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)
        
        # Display information about the action performed
        print(f"*INFO*: Contents of the repository synchronized with the workspace '{workspace}' successfully")
        print(f"*INFO*: Remember to refresh the page if the workspace does not reflect the changes")

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)

#### **📞 Power BI REST API calls 📞**

Using SemPy to perform actions through [Power BI REST API](https://learn.microsoft.com/en-us/rest/api/power-bi/) methods. 

Each function receives, at a minimum, the required values that will accompany the request. In case a workspace identifier is not provided in the call, the one corresponding to the workspace where the notebook resides is used.

The response code returned is checked for errors in the request.

---

##### 1️⃣ **Semantic model refresh**

1. **refresh_shared_semantic_model**, based on the [Datasets - Refresh Dataset In Group](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/refresh-dataset-in-group) and [Datasets - Get Refresh History In Group](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/get-refresh-history-in-group) methods

</br>

---

In [ ]:
def refresh_shared_semantic_model(dataset_id:str, workspace=None):
    """
    Author : Rafael Báguena Girbés  |   22-06-2024

    This method allows you to process a Power BI semantic model within the specified workspace. 
    It is only recommended for processing models within the shared capacity with a limitation of 8 times maximum per day.
    We check the status of the requested refresh every thirty seconds to achieve a synchronous call. 
    Otherwise the action is triggered and we have no control over how long the process takes.

    The caller must have contributor or higher workspace role.

    args:   dataset_id
                Semantic model GUID identifier, must be specified

            workspace
                Workspace name, if omitted the value of the workspace where the notebook resides is retrieved
    """
    # Check if workspace was provided, if not get current
    if workspace is None:
        workspace_id = fabric.get_workspace_id()
    else:
        workspace_id = fabric.resolve_workspace_id(workspace)

    try:
        # Initialize the client
        pbi_client = fabric.PowerBIRestClient()

        # Make the request and store the response
        response = pbi_client.post(f"/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/refreshes")

        # Error control
        if response.status_code >= 400:
            raise FabricHTTPException(response)

        # Check the status of the requested refresh
        # As long as it has no valid status, we check its evolution every 30 seconds
        refresh_status = "Not completed"
        while refresh_status not in ["Completed","Failed"]:
            time.sleep(30)
            refresh_response = pbi_client.get(f"/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/refreshes?$top=1")
            refresh_status = refresh_response.json().get('value')[0].get('status')

        # Display information about the action performed
        print(f"*INFO*: Refresh for the semantic model '{dataset_id}' successfully completed")

    except FabricHTTPException as error:
        print("*ERROR*: An exception has been found in the operation: ", error)